In [1]:
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# package imports to setup simulator experiments
import src.vsim

ModuleNotFoundError: No module named 'voting_system'

In [ ]:
help(vsim)

# Electorate

We represent our electorate as an (m,n) dimensional matrix where m corresponds to each individual voter, and n corresponds to their opinion or preference of a given topic. Topics are not actually real and should be seen a simplification of randomly generating voter preferences in a way we can easily control and adjust. To ensure we can the effect of different underlying electorate dynamics (e.g. polarisation), we can generate electorates with different skews and biases. Below is a sample in a 2D dimensional case (each voter has a unique preference on two issues) for highly homogenous clusters and heteregenous clusters.

In [13]:
# params for generation; note that given O(n^2) for NN search, it is recommended to note over do population size.
SEED = 42
ISSUES = 2
POPULATION_SIZE = 10_000

centered = vsim.electorate.setup_electorate(POPULATION_SIZE, ISSUES, scenario="centered", seed=SEED)
bipolar = vsim.electorate.setup_electorate(POPULATION_SIZE, ISSUES, scenario="bipolar", seed=SEED)
tripolar = vsim.electorate.setup_electorate(POPULATION_SIZE, ISSUES, scenario="tripolar", seed=SEED)

# same but increase diversity of intra-cluster opinion; parameterised homogenity of opinion
CLUSTER_STD = 3.5
centered_wide = vsim.electorate.setup_electorate(POPULATION_SIZE, ISSUES, scenario="centered", seed=SEED, cluster_std=CLUSTER_STD)
bipolar_wide = vsim.electorate.setup_electorate(POPULATION_SIZE, ISSUES, scenario="bipolar", seed=SEED, cluster_std=CLUSTER_STD)
tripolar_wide = vsim.electorate.setup_electorate(POPULATION_SIZE, ISSUES, scenario="tripolar", seed=SEED, cluster_std=CLUSTER_STD)

fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2,3, sharey=True, sharex=True, tight_layout=True, figsize=(14, 8))

# normal
sns.histplot(x=centered[:, 0], y=centered[:, 1], ax=ax1, label="voter")
ax1.set_title("centered - no dispersion")

sns.histplot(x=bipolar[:, 0], y=bipolar[:, 1], ax=ax2, label="voter")        
ax2.set_title("bipolar - no dispersion")

sns.histplot(x=tripolar[:, 0], y=tripolar[:, 1], ax=ax3, label="voter")
ax3.set_title("tripolar - no dispersion")

# increased dispersion
sns.histplot(x=centered_wide[:, 0], y=centered_wide[:, 1], ax=ax4, label="voter")
ax4.set_title("centered - higher dispersion")

sns.histplot(x=bipolar_wide[:, 0], y=bipolar_wide[:, 1], ax=ax5, label="voter")        
ax5.set_title("bipolar - higher dispersion")

sns.histplot(x=tripolar_wide[:, 0], y=tripolar_wide[:, 1], ax=ax6, label="voter")
ax6.set_title("tripolar - higher dispersion")

fig.suptitle("Generated Electorates", fontsize=20)
plt.show()

AttributeError: module 'src.vsim' has no attribute 'electorate'

NOTE: that our candidate preference ranges are normalised to ensure we can compare distances in a reliable way despite introducing arbitrary skewing. 

# Simulating candidates and elections

The electorate (the people) vote for candidates. Candidates can be individuals or collection of individuals (i.e. a party) but ultimately represent a given pallete of opinions. Similarly to voters, this is represented by a (m, n) matrix, where each row m corresponds to candidate m opinion on issue n.

We assume that candidates align themselves with general electorate as they want to be elected. As such, the candidate positioning on issues is a function of the electorates underlying values. Systematically, we emulate this via clustering (KMeans in our case).

In [44]:
from src import vsim

candidates = vsim.simulation.VotingSimulator.generate_candidates(n_candidates=2, electorate=bipolar_wide)
result = vsim.voting_system.NaivePlurality({}).elect(candidates=candidates, electorate=bipolar_wide)
winner = list(result.winners)[0]

AttributeError: module 'src.vsim' has no attribute 'simulation'

In [ ]:
fig, ax1 = plt.subplots(1,1, sharey=True, sharex=True, tight_layout=True, figsize=(10, 10))
sns.histplot(x=bipolar_wide[:,0], y=bipolar_wide[:,1], ax=ax1)
sns.scatterplot(x=candidates[:,0], y=candidates[:,1], label="candidates", ax=ax1, s=100, color="orange")
fig.suptitle("Candidate positioning and election winner - plurality")

arrow_props = {
    "arrowstyle": "-|>",
    "connectionstyle": "angle3",
    "lw": 1,
}

# annotate the winner of the election
plt.annotate("Election Winner (by average distance)", candidates[winner, :], xytext=[-0.015, 0.025], xycoords='data', arrowprops=arrow_props)
plt.show()

# Measuring fairness - Pluralism vs. Majority

Fairness is not well defined, even within the political science literature, but to ensure we can adequately compare different systems in a simulated setting, we have opted for a simple metric, namely the average distance from the winner(s) of the election. This average is calculated over every voter, taking the euclidian distance to the winner (or winners) of the election. If there are multiple equally weighted winners, the distance is averaged across winners. In the simple case of pluralism and majority in a non-proportional system, commonly observed in elections of the executive branch in Rebublics (simplified U.S. case without electoral votes but still). 

Pluralism is defined as the winner is the one with the most votes. Majority on the other hand requires that a givne candidate meets a threshold of 50% of the vote to be crowned winner, otherwise the election is repeated. Typically, there is a reduction in the candidate pool to ensure the elections converge on a winner; we follow suit and reduce the candidate pool by 1, removing the lowest performing candidate at each round where a definte majority cannot be crowned.

Below both are illustrated and constrated with respect to our fairness metric.